<a href="https://colab.research.google.com/github/AnyeloDev1/AnyeloDev1.github.io/blob/main/Agro_Synapsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#!/usr/bin/env python

# make sure to install these packages before running:
!pip install pandas
!pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("www.datos.gov.co", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(www.datos.gov.co,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("ch4u-f3i5", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [9]:
print(results_df)

     secuencial fecha_de_an_lisis departamento             municipio  \
0             1         7/01/2014       NARIÑO  SAN ANDRÉS DE TUMACO   
1             2        21/08/2014        HUILA           SANTA MARÍA   
2             3        22/08/2014    ANTIOQUIA              LIBORINA   
3             4        22/08/2014    ANTIOQUIA              LIBORINA   
4             5        22/08/2014    ANTIOQUIA              LIBORINA   
...         ...               ...          ...                   ...   
1995       1996        16/06/2014    ANTIOQUIA            ENTRERRÍOS   
1996       1997        16/06/2014    ANTIOQUIA            ENTRERRÍOS   
1997       1998        16/06/2014    ANTIOQUIA            ENTRERRÍOS   
1998       1999        16/06/2014    ANTIOQUIA            ENTRERRÍOS   
1999       2000        16/06/2014    ANTIOQUIA            ENTRERRÍOS   

             cultivo          estado tiempo_de_establecimiento topografia  \
0          No Indica       No indica                 No in

In [11]:
import plotly.express as px
import pandas as pd

# Ensure results_df is available and not empty from previous steps
# Using results_df as df was reported empty in kernel state, but results_df has data

if 'results_df' in globals() and not results_df.empty:
    # Convert 'potasio_intercambiable' to numeric, coercing errors
    results_df['potasio_intercambiable'] = pd.to_numeric(results_df['potasio_intercambiable'], errors='coerce')

    # Drop rows where essential columns for this visualization are missing
    df_viz = results_df.dropna(subset=['departamento', 'potasio_intercambiable', 'cultivo']).copy()

    if df_viz.empty:
        print("No hay datos suficientes para generar el gráfico después de la limpieza para visualización.")
    else:
        # Group by 'departamento' and calculate the average potassium
        avg_potassium_by_dept = df_viz.groupby('departamento')['potasio_intercambiable'].mean().reset_index()
        avg_potassium_by_dept = avg_potassium_by_dept.sort_values(by='potasio_intercambiable', ascending=False)

        # Identify the most common crop for each department
        def get_most_common_crop(series):
            return series.mode()[0] if not series.mode().empty else 'No Indica'

        most_common_crop_by_dept = df_viz.groupby('departamento')['cultivo'].apply(get_most_common_crop).reset_index()
        most_common_crop_by_dept.rename(columns={'cultivo': 'cultivo_mas_comun'}, inplace=True)

        # Merge the two dataframes
        viz_data = pd.merge(avg_potassium_by_dept, most_common_crop_by_dept, on='departamento')

        # Select the top N departments for visualization
        top_n = 10 # You can adjust this number
        viz_data_top = viz_data.head(top_n)

        # Create the bar chart
        fig = px.bar(
            viz_data_top,
            x='departamento',
            y='potasio_intercambiable',
            color='potasio_intercambiable',
            color_continuous_scale=px.colors.sequential.Plasma,
            labels={
                'departamento': 'Departamento',
                'potasio_intercambiable': 'Potasio Intercambiable Promedio (meq/100g)'
            },
            title=f'Top {top_n} Departamentos con Mayor Potasio en Suelo y Cultivo más Común',
            hover_data={'cultivo_mas_comun': True, 'potasio_intercambiable': ':.2f'}
        )

        fig.update_traces(marker_line_width=0, selector=dict(type='bar')) # Remove bar borders for cleaner look
        fig.update_layout(xaxis_tickangle=-45)
        fig.show()

else:
    print("El DataFrame 'results_df' no está disponible o está vacío. Asegúrate de que los datos de la API se cargaron correctamente.")

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

# Aseguramos que viz_data_top esté disponible del paso anterior
# (ya que está en el estado del kernel)

if 'viz_data_top' in globals() and not viz_data_top.empty:
    # Definir características (X) y variable objetivo (y)
    X = viz_data_top[['departamento', 'cultivo_mas_comun']]
    y = viz_data_top['potasio_intercambiable']

    # Preprocesamiento: Codificación One-Hot para características categóricas
    # Creamos un ColumnTransformer para aplicar OneHotEncoder a las columnas categóricas
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), ['departamento', 'cultivo_mas_comun'])
        ],
        remainder='passthrough' # Mantiene otras columnas (ninguna en este caso)
    )

    # Aplicar preprocesamiento
    X_processed = preprocessor.fit_transform(X)

    # Dado que el dataset es muy pequeño (10 muestras), la división de prueba podría ser muy limitada.
    # Para demostración, procederemos con una pequeña división, pero se reconocen las limitaciones.
    if len(viz_data_top) > 1: # Necesitamos al menos 2 muestras para train_test_split
        X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=42)

        # Inicializar y entrenar el modelo de Regresión Lineal
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Realizar predicciones
        y_pred = model.predict(X_test)

        # Evaluar el modelo
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print("Modelo de Regresión Lineal entrenado con datos del gráfico de barras.")
        print(f"Error Cuadrático Medio (MSE) en el conjunto de prueba: {mse:.4f}")
        print(f"Coeficiente de Determinación (R^2) en el conjunto de prueba: {r2:.4f}")

        # Mostrar coeficientes (opcional, útil para interpretación)
        feature_names = preprocessor.get_feature_names_out()
        print("\nCoeficientes del modelo:")
        for i, coef in enumerate(model.coef_):
            print(f"{feature_names[i]}: {coef:.4f}")
        print(f"Intercepto: {model.intercept_:.4f}")

    else:
        print("No hay suficientes datos en 'viz_data_top' para entrenar el modelo de regresión lineal.")

else:
    print("El DataFrame 'viz_data_top' no está disponible o está vacío. Asegúrate de que el gráfico de barras se generó correctamente.")

Modelo de Regresión Lineal entrenado con datos del gráfico de barras.
Error Cuadrático Medio (MSE) en el conjunto de prueba: 0.0173
Coeficiente de Determinación (R^2) en el conjunto de prueba: -0.0013

Coeficientes del modelo:
cat__departamento_ANTIOQUIA: 0.0000
cat__departamento_ATLÁNTICO: -0.0731
cat__departamento_BOLÍVAR: 0.0497
cat__departamento_BOYACÁ: -0.0002
cat__departamento_CUNDINAMARCA: 0.0027
cat__departamento_CÓRDOBA: 0.0942
cat__departamento_HUILA: 0.0000
cat__departamento_LA GUAJIRA: 0.0142
cat__departamento_RISARALDA: 0.0000
cat__departamento_VALLE DEL CAUCA: -0.0875
cat__cultivo_mas_comun_Aguacate: -0.0875
cat__cultivo_mas_comun_Frijol: 0.0497
cat__cultivo_mas_comun_Gmelina Arborea: 0.0942
cat__cultivo_mas_comun_Granadilla: 0.0000
cat__cultivo_mas_comun_Melón: -0.0589
cat__cultivo_mas_comun_Mora: 0.0027
cat__cultivo_mas_comun_No Indica: 0.0000
cat__cultivo_mas_comun_Papa Criolla: 0.0000
cat__cultivo_mas_comun_Pastos: -0.0002
Intercepto: 0.5463


In [14]:
import plotly.express as px
import pandas as pd

# Aseguramos que viz_data_top esté disponible del paso anterior
if 'viz_data_top' in globals() and not viz_data_top.empty:
    # Agrupar por cultivo_mas_comun y sumar el potasio_intercambiable para las proporciones
    # Esto nos dice qué proporción del total de potasio en los top N departamentos
    # está asociada con cada tipo de cultivo más común.
    potassium_by_crop = viz_data_top.groupby('cultivo_mas_comun')['potasio_intercambiable'].sum().reset_index()

    # Crear el gráfico circular interactivo
    fig_pie = px.pie(potassium_by_crop,
                     values='potasio_intercambiable',
                     names='cultivo_mas_comun',
                     title='Distribución del Potasio Intercambiable por Cultivo más Común en Top Departamentos',
                     labels={'potasio_intercambiable': 'Potasio Total (meq/100g)', 'cultivo_mas_comun': 'Cultivo más Común'},
                     hole=0.3)

    fig_pie.update_traces(textposition='inside', textinfo='percent+label')
    fig_pie.update_layout(showlegend=True) # Mostrar leyenda para cultivos
    fig_pie.show()

else:
    print("El DataFrame 'viz_data_top' no está disponible o está vacío. Asegúrate de que el gráfico de barras se generó correctamente antes.")